<a href="https://colab.research.google.com/github/mikeonly/nma_easy_cabbage/blob/master/success_trial_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/My Drive/Colab Notebooks/NMA

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/NMA


In [ ]:
import os, requests

#repurposed from load_steinmetz_descision.ipynb
fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

#repurposed from load_steinmetz_extra.ipynb
fname = []
fname = ['steinmetz_st.npz']

url = ["https://osf.io/4bjns/download"]

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [ ]:
import numpy as np
import pickle

alldat = np.array([])
for j in range(3):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

dat_ST = np.load('steinmetz_st.npz', allow_pickle=True)['dat']

In [ ]:
b_label_dict = {}
for s_no in range(0, len(alldat)):
  for n_no in range (0, len(dat_ST[s_no]['ss'])):
    if s_no not in b_label_dict:
      b_label_dict[s_no] = {}
    
    b_label_dict[s_no][n_no] =  alldat[s_no]['brain_area'][n_no]

In [ ]:
with open("loc_labels.pickle", "wb") as f:
  pickle.dump(b_label_dict, f)

In [ ]:

#store indices of pos and neg feedback as (session,trial)
pos_fback = []
neg_fback = []

for session in range(0, len(alldat)):
  for trial in range(0, len(alldat[session])):
    if alldat[session]['feedback_type'][trial] == -1:
      neg_fback.append((session,trial))
    # if alldat[session]['feedback_type'][trial] == +1:
    #   pos_fback.append((session,trial))

isi_p_track = []
isi_n_track = []
isi_p = []
isi_n = []
for tup in pos_fback:
  s_no = tup[0] #session number
  t_no = tup[1] #trial number
  n_no = len(dat_ST[s_no]['ss']) #total neurons
  for n in range(n_no):
    spikes = dat_ST[s_no]['ss'][n][t_no]
    if len(spikes ) != 0:
      n_isi = np.diff(spikes)
      isi_p.append(list(n_isi))
      b_region = alldat[s_no]['brain_area'][n]
      isi_p_track.append((s_no, t_no, n, b_region))


for tup in neg_fback:
  s_no = tup[0] #session number
  t_no = tup[1] #trial number
  n_no = len(dat_ST[s_no]['ss']) #total neurons
  for n in range(n_no):
    spikes = dat_ST[s_no]['ss'][n][t_no]
    if len(spikes ) != 0:
      n_isi = np.diff(spikes)
      isi_n.append(list(n_isi))
      b_region = alldat[s_no]['brain_area'][n]
      isi_n_track.append((s_no, t_no, n, b_region))



In [ ]:
with open("pos_processed_data.pickle","wb") as f:
  pickle.dump(isi_p,f)

In [ ]:
with open("neg_isi.pickle","wb") as f:
  pickle.dump(isi_n,f)

In [ ]:
with open("neg_isi_track.pickle","wb") as f:
  pickle.dump(isi_n_track,f)

In [ ]:
with open("pos_isi_track.pickle","wb") as f:
  pickle.dump(isi_p_track,f)